In [1]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline


In [2]:
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from config import Config

def split_documents(file_path, page_start=None, page_end=None, chunk_size=Config.SPLITTER_CHUNK_SIZE, chunk_overlap=Config.SPLITTER_CHUNK_OVERLAP):
    if file_path.endswith('.pdf'):
        loader = PyMuPDFLoader(file_path=file_path)
        doc = loader.load()[page_start:page_end]
    else:
        loader = TextLoader(file_path)
        docs = loader.load()
        doc = [Document(page) for page in docs[0].page_content.split('\n-----\n')[page_start:page_end]]

    if page_start is None:
        page_start = 0
    if page_end is None:
        page_end = len(doc)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    docs = text_splitter.split_documents(doc)
    return docs


In [3]:
file_path = "documentos/acordaos/0600012-49_REl_28052024_1.txt"
start_page = 2
end_page = 5
doc = split_documents(file_path, start_page, end_page)

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

from llms import get_llama

llm = get_llama()
embeddings_model = OllamaEmbeddings(
    base_url=Config.OLLAMA_EMBEDDINGS_BASE_URL,
    model=Config.OLLAMA_EMBEDDINGS_MODEL,
)

vectorstore = FAISS.from_documents(documents=doc, embedding=embeddings_model)
retriever = vectorstore.as_retriever()

In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:
    Responda com o texto simplificado do acórdão, e nada mais.

    ### FORMATO
    ```
    **Relatório (O Caso)**: Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

    Informações iniciais do processo analisado pelo juiz: indique o que o autor do recurso pediu e o que o réu alegou para se defender.
    Decisão do juiz no processo inicial: Apresente a decisão do juiz no processo inicial, descrevendo o que o juiz decidiu e as justificativas legais usadas.
    Artigos de lei e fundamentos jurídicos relevantes: cite artigos de lei e fundamentos jurídicos relevantes..
    Quem recorreu e o que alegou: indique quem recorreu à decisão e o que alegou para recorrer.

    Caixa de texto explicativa com termos jurídicos relevantes para a compreensão do assunto principal: forneça definições e explicações simples de termos, expressões ou assuntos jurídicos relevantes para a compreensão do assunto principal. Exemplo: “Propaganda antecipada negativa: A propaganda eleitoral antecipada negativa acontece quando, antes de 16 de agosto do ano eleitoral (art. 36 da Lei nº 9.504/1997), alguém faz críticas para prejudicar adversários políticos e influenciar eleitores. Essa prática é proibida e pode resultar em multa”.
    ```
    ### FIM DO FORMATO

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO
    """
)

stuff_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, stuff_chain)

chunks = []
for chunk in retrieval_chain.stream({"input": "Comece."}):
    answer = chunk.get('answer', '')
    chunks.append(answer)
    print(answer, end='', flush=True)
print()

**Relatório (O Caso)**: O caso envolve um recurso eleitoral apresentado pelo candidato Ilomar, que alega que sua pré-candidatura foi prejudicada pela realização de propaganda eleitoral antecipada. O juiz decidiu que o ato do candidato Ilomar foi caracterizado como propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, da Lei 9.504/97.

**Informações iniciais do processo**: O autor do recurso pediu a anulação da decisão do juiz que considerou a propaganda eleitoral antecipada e a aplicação da penalidade. O réu alegou que não havia pedido explícito de votos ou conduta que prejudicasse a igualdade entre os candidatos.

**Decisão do juiz no processo inicial**: O juiz decidiu que o ato do candidato Ilomar foi caracterizado como propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, da Lei 9.504/97.

**Artigos de lei e fundamentos jurídicos relevantes**: O artigo 36 da Lei 9.504/97 proíbe a propaganda eleitoral antecipada, que é definida como a realiza

In [8]:
res = "".join(chunks)
summary = res
summary

'**Relatório (O Caso)**: O caso envolve um recurso eleitoral apresentado pelo candidato Ilomar, que alega que sua pré-candidatura foi prejudicada pela realização de propaganda eleitoral antecipada. O juiz decidiu que o ato do candidato Ilomar foi caracterizado como propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, da Lei 9.504/97.\n\n**Informações iniciais do processo**: O autor do recurso pediu a anulação da decisão do juiz que considerou a propaganda eleitoral antecipada e a aplicação da penalidade. O réu alegou que não havia pedido explícito de votos ou conduta que prejudicasse a igualdade entre os candidatos.\n\n**Decisão do juiz no processo inicial**: O juiz decidiu que o ato do candidato Ilomar foi caracterizado como propaganda eleitoral antecipada e imposta a penalidade do artigo 36, § 3º, da Lei 9.504/97.\n\n**Artigos de lei e fundamentos jurídicos relevantes**: O artigo 36 da Lei 9.504/97 proíbe a propaganda eleitoral antecipada, que é definida como a 

In [3]:
from ner import NER

ner = NER()
pipeline = ner.ner

In [4]:
all_entities = [ner(chunk.page_content) for chunk in doc]

NameError: name 'doc' is not defined

In [73]:
all_entities_flattened = [entity for entities in all_entities for entity in entities]
all_entities_flattened

[{'entity': 'B-PESSOA',
  'score': 0.99925846,
  'index': 8,
  'word': 'AN',
  'start': None,
  'end': None},
 {'entity': 'B-PESSOA',
  'score': 0.9995012,
  'index': 9,
  'word': '##T',
  'start': None,
  'end': None},
 {'entity': 'B-PESSOA',
  'score': 0.9996137,
  'index': 10,
  'word': '##Ô',
  'start': None,
  'end': None},
 {'entity': 'B-PESSOA',
  'score': 0.99947196,
  'index': 11,
  'word': '##N',
  'start': None,
  'end': None},
 {'entity': 'B-PESSOA',
  'score': 0.9993319,
  'index': 12,
  'word': '##IO',
  'start': None,
  'end': None},
 {'entity': 'I-PESSOA',
  'score': 0.99985313,
  'index': 13,
  'word': 'I',
  'start': None,
  'end': None},
 {'entity': 'I-PESSOA',
  'score': 0.99988854,
  'index': 14,
  'word': '##LO',
  'start': None,
  'end': None},
 {'entity': 'I-PESSOA',
  'score': 0.99989414,
  'index': 15,
  'word': '##MA',
  'start': None,
  'end': None},
 {'entity': 'I-PESSOA',
  'score': 0.9998938,
  'index': 16,
  'word': '##R',
  'start': None,
  'end': None}

In [70]:
import pprint


pprint.pprint(all_entities)

[[{'end': None,
   'entity': 'B-PESSOA',
   'index': 8,
   'score': 0.99925846,
   'start': None,
   'word': 'AN'},
  {'end': None,
   'entity': 'B-PESSOA',
   'index': 9,
   'score': 0.9995012,
   'start': None,
   'word': '##T'},
  {'end': None,
   'entity': 'B-PESSOA',
   'index': 10,
   'score': 0.9996137,
   'start': None,
   'word': '##Ô'},
  {'end': None,
   'entity': 'B-PESSOA',
   'index': 11,
   'score': 0.99947196,
   'start': None,
   'word': '##N'},
  {'end': None,
   'entity': 'B-PESSOA',
   'index': 12,
   'score': 0.9993319,
   'start': None,
   'word': '##IO'},
  {'end': None,
   'entity': 'I-PESSOA',
   'index': 13,
   'score': 0.99985313,
   'start': None,
   'word': 'I'},
  {'end': None,
   'entity': 'I-PESSOA',
   'index': 14,
   'score': 0.99988854,
   'start': None,
   'word': '##LO'},
  {'end': None,
   'entity': 'I-PESSOA',
   'index': 15,
   'score': 0.99989414,
   'start': None,
   'word': '##MA'},
  {'end': None,
   'entity': 'I-PESSOA',
   'index': 16,
   '

In [74]:
words = [[ent['word'], ent['entity']] for ent in all_entities_flattened]
print(list(words))

[['AN', 'B-PESSOA'], ['##T', 'B-PESSOA'], ['##Ô', 'B-PESSOA'], ['##N', 'B-PESSOA'], ['##IO', 'B-PESSOA'], ['I', 'I-PESSOA'], ['##LO', 'I-PESSOA'], ['##MA', 'I-PESSOA'], ['##R', 'I-PESSOA'], ['V', 'I-PESSOA'], ['##AS', 'I-PESSOA'], ['##CO', 'I-PESSOA'], ['##N', 'I-PESSOA'], ['##CE', 'I-PESSOA'], ['##LO', 'I-PESSOA'], ['##S', 'I-PESSOA'], ['CR', 'I-PESSOA'], ['##U', 'I-PESSOA'], ['##Z', 'I-PESSOA'], ['F', 'B-PESSOA'], ['##RA', 'B-PESSOA'], ['##N', 'B-PESSOA'], ['##CI', 'B-PESSOA'], ['##SC', 'B-PESSOA'], ['##O', 'B-PESSOA'], ['J', 'I-PESSOA'], ['##AR', 'I-PESSOA'], ['##DE', 'I-PESSOA'], ['##L', 'I-PESSOA'], ['R', 'I-PESSOA'], ['##O', 'I-PESSOA'], ['##DR', 'I-PESSOA'], ['##IG', 'I-PESSOA'], ['##U', 'I-PESSOA'], ['##ES', 'I-PESSOA'], ['DE', 'I-PESSOA'], ['SO', 'I-PESSOA'], ['##US', 'I-PESSOA'], ['##A', 'I-PESSOA'], ['O', 'B-ORGANIZACAO'], ['##AB', 'B-ORGANIZACAO'], ['CA', 'B-PESSOA'], ['##VA', 'B-PESSOA'], ['##LC', 'B-PESSOA'], ['##AN', 'B-PESSOA'], ['##TE', 'B-PESSOA'], ['M', 'I-PESSOA'], 

In [75]:
full_words = list()

for word in words:
    w = word[0]
    e = word[1]
    if len(word[0]) < 3 or word[0][:2] != '##':
        full_words.append([w, e])
    else:
        full_words[-1][0] = full_words[-1][0] + w[2:]

print(full_words)

[['ANTÔNIO', 'B-PESSOA'], ['ILOMAR', 'I-PESSOA'], ['VASCONCELOS', 'I-PESSOA'], ['CRUZ', 'I-PESSOA'], ['FRANCISCO', 'B-PESSOA'], ['JARDEL', 'I-PESSOA'], ['RODRIGUES', 'I-PESSOA'], ['DE', 'I-PESSOA'], ['SOUSA', 'I-PESSOA'], ['OAB', 'B-ORGANIZACAO'], ['CAVALCANTE', 'B-PESSOA'], ['MENDONÇA', 'I-PESSOA'], ['VIEIRA', 'I-PESSOA'], ['OAB', 'B-ORGANIZACAO'], ['MINISTÉRIO', 'B-ORGANIZACAO'], ['PÚBLICO', 'I-ORGANIZACAO'], ['ELEITORAL', 'I-ORGANIZACAO'], ['ANTÔNIO', 'B-PESSOA'], ['ILOMAR', 'I-PESSOA'], ['VASCONCELOS', 'I-PESSOA'], ['CRUZ', 'I-PESSOA'], ['[UNK]', 'B-ORGANIZACAO'], ['Zona', 'I-ORGANIZACAO'], ['Canindé', 'B-LOCAL'], ['/', 'B-LOCAL'], ['CE', 'B-LOCAL'], ['art', 'B-LEGISLACAO'], ['.', 'I-LEGISLACAO'], ['36', 'I-LEGISLACAO'], [',', 'I-LEGISLACAO'], ['§', 'I-LEGISLACAO'], ['[UNK]', 'I-LEGISLACAO'], [',', 'I-LEGISLACAO'], ['da', 'I-LEGISLACAO'], ['Lei', 'I-LEGISLACAO'], ['9', 'I-LEGISLACAO'], ['.', 'I-LEGISLACAO'], ['504', 'I-LEGISLACAO'], ['/', 'I-LEGISLACAO'], ['97', 'I-LEGISLACAO'], ['

In [85]:
from functools import reduce


topics = [word for word in full_words]
full_topics = []

for word in topics:
    if word[1][0] == 'B':
        full_topics.append([word[0], word[1][2:]])
    elif word[1][0] == 'I':
        full_topics[-1][0] = full_topics[-1][0] + ' ' + word[0]

full_topics_filtered = list(filter(lambda x: x not in full_topics, full_topics))
full_topics_text = reduce(lambda x, y: x + '\n‒ ' + y[0], full_topics_filtered, '')

print(full_topics_text)

In [ ]:
missing_topics_prompt = PromptTemplate.from_template(
    """
    O resumo do acórdão está incompleto. Por favor, com base nos seguintes tópicos, complete o resumo do acórdão:

    {topics}

    ### Resumo do Acórdão
    {context}
    ### Fim do Resumo

    Resumo corrigido:
    """
)

missing_topics_chain = create_stuff_documents_chain(llm, missing_topics_prompt)
retrieval_chain = create_retrieval_chain(retriever, missing_topics_chain)

chunks = []
for chunk in retrieval_chain.stream({"topics": full_topics, "context": summary}):
    answer = chunk.get('answer', '')
    chunks.append(answer)
    print(answer, end='', flush=True)